In [1]:
# Import librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
#markdown de que tratan las columnas
#hacer un grafico de una pregunta interesante (grupo de columnas) 
#primeras 15 patrick
#siguient bruno 
#34 a final juan
#https://docs.google.com/document/d/1xeXGwWcrq0DfJiaZdMEY90f4sume7BjmDSZ1k1nbNno/edit?usp=sharing
#https://data.educacion.gob.ar/aprender-respuestas-estudiante

In [2]:
ruta_archivo_leng = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Lengua(in).csv"
ruta_archivo_mate = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Matematica(in).csv"

In [13]:
# cargar la versión CSV:
#data = pd.read_csv(ruta_archivo, sep=';', encoding='utf-8')
data = pd.read_csv(ruta_archivo, sep=';', encoding="latin-1")

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Columns: 1035 entries, jurisdiccion to Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
dtypes: object(1035)
memory usage: 9.7+ MB


# Dataset



* las columnas 12345 tratan de edad: 

In [15]:
from pandas.api.types import is_numeric_dtype

def analizar_dataframe(df):
    for col in df.columns:
        print(f"\nColumna: {col}")
        tipo = df[col].dtype

        if is_numeric_dtype(tipo):
            print("Tipo: float")
            print(f"  Media: {df[col].mean()}")
            print(f"  Moda: {df[col].mode().iloc[0] if not df[col].mode().empty else 'Sin moda'}")
            print(f"  Mínimo: {df[col].min()}")
            print(f"  Máximo: {df[col].max()}")

        elif pd.api.types.is_object_dtype(tipo):
            print("Tipo: objeto")
            num_categorias = df[col].nunique()
            if num_categorias > 20:
                print(f"  Más de 20 categorías ({num_categorias}), no se listan.")
            else:
                print(f"  Categorías más frecuentes:")
                print(f"  {df[col].value_counts().head(5)}")
        else:
            print(f"Tipo no explorado: {tipo}")
        print("\n")


In [17]:
#analizar_dataframe(data)

In [3]:
#import pandas as pd

# Cargar ambos CSV
df1 = pd.read_csv(ruta_archivo_leng, sep=';', decimal=",", encoding="latin-1")
df2 = pd.read_csv(ruta_archivo_mate, sep=';', decimal=",",  encoding="latin-1")

# Comparar dimensiones (deberían ser iguales)
print("Shape archivo1:", df1.shape)
print("Shape archivo2:", df2.shape)




/tmp/ipykernel_46523/2652677367.py:4: DtypeWarning: Columns (28,34,40,58,63,74,79,83,118,122,125,129,133,134,137,141,145,149,153,159,163,168,173,178,183,188,193,198,209,213,214,218,223,228,250,255,265,275,279,285,290,300,304,310,322,330,338,346,354,362,391,397,403,410,420,423,427,431,435,439,443,447,451,455,459,463,471,475,479,483,487,492,495,499,503,508,516,525,552,558,572,576,580,584,590,597,604,611,618,632,637,642,647,652,657,660,664,668,672,676,680,684,688,692,696,700,705,709,712,716,720,724,728,732,736,740,745,750,755,760,765,770,775,786,791,796,800,806,811,816,821,825,831,836,840,845,851,856,860,866,871,876,881,885,891,895,896,901,905,915,925,929,932,936,940,944,948,952,956,960,965,970,977,986,992,996) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(ruta_archivo_leng, sep=';', decimal=",", encoding="latin-1")


Shape archivo1: (1228, 1035)
Shape archivo2: (1228, 1035)


/tmp/ipykernel_46523/2652677367.py:5: DtypeWarning: Columns (28,34,40,58,63,74,79,83,118,122,125,129,133,134,137,141,145,149,153,159,163,168,173,178,183,188,193,198,209,213,214,218,223,228,250,255,265,275,279,285,290,300,304,310,322,330,338,346,354,362,391,397,403,410,420,423,427,431,435,439,443,447,451,455,459,463,471,475,479,483,487,492,495,499,503,508,516,525,552,558,572,576,580,584,590,597,604,611,618,632,637,642,647,652,657,660,664,668,672,676,680,684,688,692,696,700,705,709,712,716,720,724,728,732,736,740,745,750,755,760,765,770,775,786,791,796,800,806,811,816,821,825,831,836,840,845,851,856,860,866,871,876,881,885,891,895,896,901,905,915,925,929,932,936,940,944,948,952,956,960,965,977,986,992,996) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(ruta_archivo_mate, sep=';', decimal=",",  encoding="latin-1")


In [24]:
print(df1.columns)

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Persona_Resp_Blanco', 'Nivel_Ed_Persona_Resp_No_disponible',
       'Nivel_Ed_Persona_Resp_Multimarca',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleto',
       'Nivel_Ed_Persona_Resp_Primaria_completo',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleto',
       'Nivel_Ed_Persona_Resp_Secundaria_completo',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1035)


In [28]:
# Listar las columnas de cada uno
cols1 = set(df1.columns)
cols2 = set(df2.columns)

#print("Columnas en archivo1:", cols1)
#print("Columnas en archivo2:", cols2)

# Columnas que comparten
comunes = cols1.intersection(cols2)
#print("\n✅ Columnas comunes:", comunes)

# Columnas que están en uno pero no en el otro
solo_en_1 = cols1 - cols2
solo_en_2 = cols2 - cols1

#print("\n📌 Solo en archivo1:", solo_en_1)
#print("📌 Solo en archivo2:", solo_en_2)

In [5]:
df1.head(5)

,NSE_Quintiles_Q1,NSE_Quintiles_Q2,NSE_Quintiles_Q3,NSE_Quintiles_Q4,NSE_Quintiles_Q5,Nivel_Ed_Madre_Blanco,Nivel_Ed_Madre_Multimarca,Nivel_Ed_Madre_No_disponible,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completo,...,repitencia_Repitió_1_vez,repitencia_Repitió_2_veces_o_más,repitencia__Blanco,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años
0,"305,4540767","395,8163098","448,5434023","406,6948799","447,7226802",,,,"14,4168609","110,0668966",...,"357,5814759","83,9450754",,Estatal,"354,2711408","147,9215083","74,4798765",,1412.571715,"14,9871081"
1,"90,8504281","141,2491195","248,7429795","343,8960813","515,7330062",,,,"6,2177038","38,3151466",...,"111,4316556","23,378234",,Privado,"129,5492375","21,2352027","8,1378118",,1169.178819,"12,3705437"
2,"155,5140104","163,5532949","241,3056548","262,5971194","214,4821643",,,,"6,4459497","53,343938",...,"187,7095467","31,4840438",,Estatal,"200,8463227","67,2922915","14,4204609",,743.266667,"11,6265022"
3,"43,3365819","97,3160697","201,7942854","359,6517533","616,6067874",,,,"1,3290809","23,5706013",...,"80,0875317","12,855258",,Privado,"99,0596264","14,6733387","10,8004209",,1190.290836,"3,8812557"
4,"135,5338108","151,9950071","161,2851543","229,7151767","136,1156421",,,,"7,3513511","47,6269065",...,"157,004763","17,6874718",,Estatal,"148,2518419","35,0811324","32,0995373",,597.54294,"1,6693392"


In [6]:
df2.head(5)

,NSE_Quintiles_Q1,NSE_Quintiles_Q2,NSE_Quintiles_Q3,NSE_Quintiles_Q4,NSE_Quintiles_Q5,Nivel_Ed_Madre_Blanco,Nivel_Ed_Madre_Multimarca,Nivel_Ed_Madre_No_disponible,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completo,...,repitencia_Repitió_1_vez,repitencia_Repitió_2_veces_o_más,repitencia__Blanco,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años
0,"311,9128657","397,5061885","434,6976716","412,2875671","444,6841645",,,,"14,3109414","112,0495363",...,"358,2600348","83,2384833",,Estatal,"348,0311521","150,3783865","72,6289195",,1416.119063,"13,9309367"
1,"89,5409038","139,9633298","247,5290802","350,3724089","512,3370187",,,,"6,3800858","38,3987288",...,"115,2950114","21,7121787",,Privado,"131,4566553","21,7252296","8,3034349",,1165.723556,"12,5338653"
2,"149,7354965","158,8934206","249,4876789","267,8733001","215,2577947",,,,"5,1814754","47,4528237",...,"200,4127712","29,0833523",,Estatal,"205,2399143","69,4542603","14,9892824",,739.068803,"12,4954312"
3,"43,4161772","93,910606","209,6465822","357,4446085","613,570459",,,,"1,2791598","22,4277114",...,"77,002925","12,9283674",,Privado,"92,6884427","14,9056583","10,8398263",,1193.447147,"6,1073591"
4,"136,4632764","149,6396907","157,4560776","222,8633378","131,1282724",,,,"7,3064068","43,7705288",...,"151,4206304","15,3946736",,Estatal,"133,1047668","36,2340189","30,4185812",,594.776443,"3,016845"


In [4]:

# Columnas comunes (convertir a lista ordenada)
comunes = sorted(set(df1.columns).intersection(set(df2.columns)))

#print("Columnas comunes:", comunes)

# Filtrar ambos dataframes solo con las columnas comunes
df1 = df1[comunes]
df2 = df2[comunes]

#df1 = df1.reset_index(drop=True)
#df2 = df2.reset_index(drop=True)
# Ver en qué filas/columnas difieren
diff = (df1 != df2)

# Contar cuántas celdas son distintas
total_diferencias = diff.sum().sum()
#print("Total de diferencias:", total_diferencias)

# Mostrar las filas con diferencias
diferencias = df1[diff.any(axis=1)]
#print("Filas con diferencias:")
#print(diferencias)

# Si querés ver lado a lado:
comparacion = df1.compare(df2)
print("Comparacion")
print(comparacion)


Comparacion
     NSE_Quintiles_Q1              NSE_Quintiles_Q2               \
                 self        other             self        other   
0         305,4540767  311,9128657      395,8163098  397,5061885   
1          90,8504281   89,5409038      141,2491195  139,9633298   
2         155,5140104  149,7354965      163,5532949  158,8934206   
3          43,3365819   43,4161772       97,3160697    93,910606   
4         135,5338108  136,4632764      151,9950071  149,6396907   
...               ...          ...              ...          ...   
1223       129,747947  119,5181099      230,4066794  231,8773807   
1224        5,5134833   5,68924228       30,3344758   29,6580944   
1225       17,1091029   17,8230603        16,485628   17,6269966   
1226       98,6424357  101,0602407      216,9404383  212,5386855   
1227        11,974876   10,8913794       15,8562432   14,8733105   

     NSE_Quintiles_Q3              NSE_Quintiles_Q4               \
                 self        other 